# 🧠 고전 CNN 모델 실습: LeNet-5 / VGG16

이번 노트북에서는 고전적인 CNN 모델 3종을 실습합니다.

1. **LeNet-5** — MNIST 직접 학습 및 평가
3. **VGG16** — ImageNet 사전학습 가중치로 예측 실습

모든 모델의 파라미터 수를 비교하고, 구조를 살펴봅니다.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, utils
from tensorflow.keras.applications import VGG16, MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
import matplotlib.pyplot as plt
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.20.0


## [1] LeNet-5 구현 및 MNIST 학습

In [2]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print('Train:', x_train.shape, ' Test:', x_test.shape)

Train: (60000, 28, 28, 1)  Test: (10000, 28, 28, 1)


In [3]:
def build_lenet5():
    model = models.Sequential([
        layers.Conv2D(6, (5,5), activation='tanh', input_shape=(28,28,1), padding='same'),
        layers.AveragePooling2D(pool_size=(2,2)),
        layers.Conv2D(16, (5,5), activation='tanh'),
        layers.AveragePooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dense(120, activation='tanh'),
        layers.Dense(84, activation='tanh'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

lenet5 = build_lenet5()
lenet5.summary()

C:\Users\kimyo\anaconda3\envs\PR\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 14, 14, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 5, 5, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 120)            │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,706 (241.04 KB)

 Trainable params: 61,706 (241.04 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
history = lenet5.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.1, verbose=2)

Epoch 1/5
422/422 - 3s - 8ms/step - accuracy: 0.8989 - loss: 0.3482 - val_accuracy: 0.9670 - val_loss: 0.1272
Epoch 2/5
422/422 - 2s - 5ms/step - accuracy: 0.9623 - loss: 0.1264 - val_accuracy: 0.9767 - val_loss: 0.0816
Epoch 3/5
422/422 - 2s - 6ms/step - accuracy: 0.9745 - loss: 0.0837 - val_accuracy: 0.9785 - val_loss: 0.0708
Epoch 4/5
422/422 - 2s - 5ms/step - accuracy: 0.9814 - loss: 0.0616 - val_accuracy: 0.9833 - val_loss: 0.0564
Epoch 5/5
422/422 - 2s - 5ms/step - accuracy: 0.9854 - loss: 0.0477 - val_accuracy: 0.9833 - val_loss: 0.0534


In [5]:
test_loss, test_acc = lenet5.evaluate(x_test, y_test, verbose=0)
print(f"LeNet-5 테스트 정확도: {test_acc:.4f}")

LeNet-5 테스트 정확도: 0.9831


## [3] VGG16 (ImageNet 사전학습 가중치)

In [6]:
vgg16 = VGG16(weights='imagenet')
vgg16.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
img_path = tf.keras.utils.get_file('elephant.jpg', 'https://cdn.hackaday.io/images/1219231493836853483.jpg')
img_vgg = image.load_img(img_path, target_size=(224, 224))
x_vgg = image.img_to_array(img_vgg)
x_vgg = np.expand_dims(x_vgg, axis=0)
x_vgg = preprocess_input(x_vgg)

preds_vgg = vgg16.predict(x_vgg)
print('VGG16 예측 결과:', decode_predictions(preds_vgg, top=3)[0])

15881/15881 ━━━━━━━━━━━━━━━━━━━━ 0s 8us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
VGG16 예측 결과: [('n02504458', 'African_elephant', np.float32(0.84337336)), ('n01871265', 'tusker', np.float32(0.15428616)), ('n02504013', 'Indian_elephant', np.float32(0.0023405228))]


## [4] 모델별 파라미터 수 비교

In [9]:
models_info = {
    'LeNet-5 (MNIST)': lenet5.count_params(),
    'VGG16 (ImageNet)': vgg16.count_params()
}
for name, params in models_info.items():
    print(f'{name}: {params:,} parameters')

LeNet-5 (MNIST): 61,706 parameters
VGG16 (ImageNet): 138,357,544 parameters
